---
**Name: Sameer Koleshwar**<br>
**Roll no: 202SP010**

---

In [9]:
#import reuired libraries
import numpy as np
import time
from numpy import fft
import cmath
import math
import copy

---
**Q2 (a) Write python from scratch for computing 2D DFT{X(k,l)} of the following 2D array**
<br>**(i) x(m,n) = np.array([[1, 0],[2, 1]])**
<br>**(ii) x(m,n) = np.array([[1,2, 3,4], [5, 6, 7, 8], [9,10,11,12], [13,14,15,16]])**

---

In [2]:
x1 = np.array([[1, 0],[2, 1]])
x2 = np.array([[1,2, 3,4], [5, 6, 7, 8], [9,10,11,12], [13,14,15,16]])

In [3]:
def dft_2d(data):
    
    """
    DFT of 2d data
    Input
        data: 2d array 
    Output
        A 2D array respresents the DFT of input data
    """
    
    # read no. of rows and cols of input data, and create a empty array with same dimension
    # as of input data with complex as a data type
    M, N = data.shape 
    fd = np.empty([M, N], dtype=complex)

    # There will 2 for-loops(k, l) to calculate the DFT of each pixel of 2D array,
    
    for k in range(M):
        for l in range(N):
            
            # To calculate DFT of 1 pixel we have to multiply the exponent terms with every pixel of input data
            # Here, we will form a 2D exponents co-efficients matrix named as tf
            tf = np.empty([M, N], dtype=complex)
            for m in range(0, M):
                for n in range(0, N):
                    temp = (m*k/M) + (n*l/N)
                    tf[m, n] = cmath.exp(complex(0, -2*np.pi*temp))
            #print(tf)
            
            # To do the double summation operation on input data multiplied with exponent co-efficients terms 
            fd[k, l] = np.sum(np.multiply(tf, data))
    return np.around(fd)

In [4]:
print("2D DFT from scratch:")
t = time.time()
print(dft_2d(x1))
print("Time required: {} sec".format(time.time() - t))
print("\n")
print("DFT using fft package")
t = time.time()
print(fft.fft2(x1))
print("Time required: {} sec".format(time.time() - t))

2D DFT from scratch:
[[ 4.+0.j  2.-0.j]
 [-2.-0.j  0.+0.j]]
Time required: 0.0 sec


DFT using fft package
[[ 4.+0.j  2.+0.j]
 [-2.+0.j  0.+0.j]]
Time required: 0.016001224517822266 sec


In [5]:
print("2D DFT from scratch:")
t = time.time()
print(dft_2d(x2))
print("Time required: {} sec".format(time.time() - t))
print("\n")
print("DFT using fft package")
t = time.time()
print(fft.fft2(x2))
print("Time required: {} sec".format(time.time() - t))

2D DFT from scratch:
[[136. +0.j  -8. +8.j  -8. -0.j  -8. -8.j]
 [-32.+32.j   0. +0.j   0. +0.j  -0. +0.j]
 [-32. -0.j   0. +0.j   0. +0.j  -0. +0.j]
 [-32.-32.j  -0. +0.j  -0. +0.j  -0. +0.j]]
Time required: 0.0 sec


DFT using fft package
[[136. +0.j  -8. +8.j  -8. +0.j  -8. -8.j]
 [-32.+32.j   0. +0.j   0. +0.j   0. +0.j]
 [-32. +0.j   0. +0.j   0. +0.j   0. +0.j]
 [-32.-32.j   0. +0.j   0. +0.j   0. +0.j]]
Time required: 0.01561427116394043 sec


---
**Q.3. (a) Write python from scratch for 2D Circular convolution using  Doubly Block Circulant matrices method between**

**input=np.array([[1,2,3],[4,5,6],[7,8,9]])and filter=np.array([[1,2,1],[0,0,0],[-1,-2,-1]])**

---

In [6]:
def getH1(a, n, m):
    
    """
    This function will return a circulant matrix of size (n,m) from the a row-vector input 'a'
    Input
        a: 1d array
        n, m: int
    Output:
        H1: numpy nd-array(2d), circulant matrix formed from 'a'.
    """
    H1 = np.zeros([n, m])
    for i in range(m):
        H1[:, i] = a        # In the 1st column copy 'a' vector as it is, in next iteration copy the shifted 'a'
        a = np.roll(a, 1)   # used roll funtion for shifting 1 element 
    return H1

def get_circulant(data, kernel):
    
    """
    This function will calculate the H matrix for given kernel using dimension of input data
    Input:
        data, kernel: 2D array
    Output:
        H matrix 
    """
    
    # get the shape of data, kernel
    m1, n1 = data.shape
    m2, n2 = kernel.shape
    
    # create a 2d array of required size, with zero padding the elements in kernel
    h = np.zeros([(m1), (n1)])
    m, n = h.shape
    h[m-m2:, 0:n2] = kernel
    
    #print("data: ", (m1, n1), "kernel:", (m2, n2), "req: ", (m,n))
    #print("h")
    #print(h)
    
    # create a 1st column of circulant matrix, which contains H1, H2, ...., Hm
    # using funtion getH1
    H_ = np.zeros([n*m, n1])
    #print("H_ shape ", H_.shape)
    #H1[n, 0:m] = H1, H1[n, m:2m] = H2 ........
    for k in range(1, m+1):
        #print("{}:{} ".format((m-k), h[m-k, :]))
        H_[(k-1)*n:k*n, :] = getH1((h[m-k, :]).T, n, n1)
       
    #print("H_\n")
    #print(H_)
    #print(H_.shape)
    
    H = np.zeros([n*m, m1*n1])
    H[:, 0:n1] = H_
    
    mH_, nH_ = H_.shape
    #print("H", H.shape)
    
    # using this loop extend the remaining columns of H matrix
    a = copy.copy(H_)
    for j in range(1, m1):
        a = np.roll(a, m1*n1)
        H[:, (j*n1):(j+1)*n1] = a
        
    #print("H\n")
    #print(H)
    return H

    
def conv2D_circulant(data, kernel):
    
    """
    This function will do the multiplication of H and f, i.e. g = H.f
    Input
        data, kernel
    Output:
        g: 2d array, circular convolution of input data and kernel using doubly circulant matrix
    """
    
    # get the shape of data, kernel
    m1, n1 = data.shape
    m2, n2 = kernel.shape
    #m, n = (m1+m2-1), (n1+n2-1)
    
    # prepre the f vector 
    f = np.reshape(np.flipud(data), -1)
    #print("\n")
    #print(f)
    
    # get doubly circulant matrix for given kernel
    H = get_circulant(data, kernel)
    
    # convolution
    g = np.dot(H, f)
    g = np.flipud(np.reshape(g, [m1, n1]))
    #print("g\n")
    #print(g)
    

    return g


In [7]:
data = np.array([[1,2,3],[4,5,6],[7,8,9]])
kernel = np.array([[1,2,1],[0,0,0],[-1,-2,-1]])

print("Cicular convolution using doubly circulant matrix from scratch: ")
t = time.time()
print(conv2D_circulant(data, kernel ))
print("time required: ", time.time()-t)

Cicular convolution using doubly circulant matrix from scratch: 
[[ 24.  24.  24.]
 [-12. -12. -12.]
 [-12. -12. -12.]]
time required:  0.015480518341064453


In [8]:
# Testing the circular convolution function on problem discussed in the class
data = np.array([[0, 1, 0],[1, 3, -1],[1, 2, 1]])
kernel = np.array([[1, 0], [1, -1]])

print("Cicular convolution using doubly circulant matrix from scratch: ")
t = time.time()
print(conv2D_circulant(data, kernel ))
print("time required: ", time.time()-t)

Cicular convolution using doubly circulant matrix from scratch: 
[[ 1.  4. -2.]
 [ 3.  4. -3.]
 [ 0.  2. -1.]]
time required:  0.0
